In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
data=pd.read_csv('../input/csgo-round-winner-classification/csgo_round_snapshots.csv')


In [4]:
data

In [5]:
data.dtypes

In [6]:
np.sum(np.sum(data.isnull()))

In [7]:
data.drop(data.select_dtypes(np.number),axis=1)

In [8]:
data['bomb_planted']=data['bomb_planted'].astype(np.int16)

In [9]:
encoder=LabelEncoder()
data['map']=encoder.fit_transform(data['map'])

map_mappings={index:label for index,label in enumerate(encoder.classes_)}

In [10]:
map_mappings

In [11]:
data['round_winner']=encoder.fit_transform(data['round_winner'])

round_winner_mappings={index:label for index,label in enumerate(encoder.classes_)}

In [12]:
round_winner_mappings

In [13]:
y=data['round_winner']
x=data.drop('round_winner',axis=1)

In [14]:
scaler=RobustScaler()
x=scaler.fit_transform(x)
pd.DataFrame(x)

In [15]:
pca=PCA(n_components=84)
pca.fit(x)

In [17]:
plt.figure(figsize=(10,10))
plt.hist(pca.explained_variance_ratio_)
plt.show()

In [19]:
def getKComponents(pca,alpha):
    total_variance=0
    
    for feature,variance in enumerate(pca.explained_variance_ratio_):
        total_variance+=variance
        if (total_variance>=1-alpha):
            return feature+1
    return len(pca.explained_variance_ratio_)

In [23]:
k=getKComponents(pca,0.05)
k

In [21]:
x=pca.transform(x)[:,0:k]
pd.DataFrame(x)

In [24]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8)


In [26]:
log_model=LogisticRegression(verbose=True)
nn_model=MLPClassifier(verbose=True)

log_model.fit(x_train,y_train)
nn_model.fit(x_train,y_train)

In [28]:
print("logistic model :",log_model.score(x_test,y_test))
print("neural net model :",nn_model.score(x_test,y_test))